# ML Final Project

## Goals of the Project

This project presents an attempt to quantify the predictability of the Bundesliga football games based on the information contained in game statistics staring from season 2006/07 onwards. In particular,

* we use a deep neural network to examine the correlations between different features provided from the statistics of individual games.
* we ask the question: "to what accuracy canone determine the outcome and score of any Bundesliga game, given the games statistics from a number of previous seasons?".
* we compare the predictions of our model on the Bundesliga season 2016/17 up to the present matchday. We also make a prediction for the final Bundesliga table for season 2016/17.

Studying the predictive power of footbal statistics on the outcome of games is an interesting and challenging problem of general importance both to sports, and the development of a proper intuition about the inner workings of machine learning techniques. The difficulty of the problem is enhanced by the occurence of outlier events in the data, for example a team performing better in a given game but nevertheless losing to their opponent 'misfortunately'. Such events can easily be recognised as noise or bias by the neural network leading to an increase in both the in-sample and out-of-sample errors. 


## Data and Features

We use a manually developed football games dataset, collected from data provided at ... and kicker.de . We then modified those sets introducing some new features as combinations of the existing ones, while dropping others.

Each data point is represented as a vector containing the following features: (list names of all features below)

Last, each feature from the data was normalised to unity with respect to all games in the data set.

To acquire a better understanding of the data used to train the deep neural net, below we show histrograms of selected interesting features, that are often used intuitively to estimate the outcome of a given fixture:

1. Shots (again, two hists: H and A)

2. Shots on target (let's plot the two hists (H and A) on top of each other with some transparency)

3. Attendance

4. 5, 6....





## Model: Deep Neural Network

Our model is a deep relu neural net, consisting of an input layer, four hidden layers and an output layer:
* input layer: ?? features
* hidden layer 1: 50 neurons
* hidden layer 2: 500 neurons
* hidden layer 3: 500 neurons
* hidden layer 4: 50 neurons
* output layer with softmax activation: ?? features

We train with minibatches using the 'adam' optimiser, and minimise the categorical entropy function. Additionally, we apply 'dropout' regularisation after hidden layer 4. 

here comes the code...

## Results

show some plots here